In [1]:
from dataset import *
from tensorflow.python.framework import graph_util


train_data = DataSet('num_letter.tfrecords')
validate_data = DataSet('num_letter.tfrecords')
# train_img, train_label = data.get_batch(500)
val_img, val_label = validate_data.get_batch(500)


In [2]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.1))


def init_bias(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))


In [3]:
with tf.Graph().as_default() as g:
    x_ = tf.placeholder(tf.float32, [None, 1152], name='image')
    y = tf.placeholder(tf.float32, [None, 34], 'label')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    x = tf.reshape(x_, [-1, 48, 24, 1], name='x')

    # 第一层卷积
    w1 = init_weights([3, 3, 1, 32])
    b1 = init_bias([32])
    conv1 = tf.nn.relu(tf.nn.conv2d(x, w1,  strides=[1, 1, 1, 1], padding='SAME') + b1)  # shape=(?, 48, 24, 32)
    # 池化
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # # shape=(?, 24, 12, 32)


    # 第二层卷积
    w2 = init_weights([3, 3, 32, 64])
    b2 = init_bias([64])
    conv2 = tf.nn.relu(tf.nn.conv2d(pool1, w2,  strides=[1, 1, 1, 1], padding='SAME') + b2)  # shape=(?, 24, 12, 64)
    # 池化
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # shape=(?, 12, 6, 64)


    # 第三层卷积
    w3 = init_weights([3, 3, 64, 96])
    b3 = init_bias([96])
    conv3 = tf.nn.relu(tf.nn.conv2d(pool2, w3,  strides=[1, 1, 1, 1], padding='SAME') + b3)  # shape=(?, 12, 6, 96)
    # 池化
    pool3_ = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # shape=(?, 6, 3, 96)


    pool3 = tf.reshape(pool3_, [-1, 6*3*96])

    # 全连接层
    w4 = init_weights([6 * 3 * 96, 512])
    b4 = init_bias([512])
    h = tf.nn.relu(tf.matmul(pool3, w4) + b4)

    h = tf.nn.dropout(h, keep_prob)

    # 输出层
    w_o = init_weights([512, 34])
    b_o = init_bias([34])
    #y_o = tf.nn.softmax(tf.matmul(h, w_o) + b_o)
    y_o = tf.matmul(h, w_o) + b_o


    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_o, labels=y))
    #cost = -tf.reduce_sum(y*tf.log(y_o))
    train_op = tf.train.AdamOptimizer().minimize(cost)
    prediction = tf.equal(tf.argmax(y_o, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32), name='accuracy')


In [4]:
with tf.Session(graph=g) as sess:

    sess.run(tf.global_variables_initializer())

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    for i in range(10):
        train_accuracy, loss = sess.run(
            [accuracy, cost],
            feed_dict={x_: train_data.get_batch(500)[0], y: train_data.get_batch(500)[1], keep_prob: 1.0})
        print("step %d, accuracy: %g, loss : %g" % (i, train_accuracy, loss))
        train_op.run(feed_dict={x_: train_data.get_batch(500)[0], y: train_data.get_batch(500)[1], keep_prob: 0.5})

    for i in range(10):

        print(sess.run(accuracy, feed_dict={x_: val_img, y: val_label, keep_prob: 1.0}))

    coord.request_stop()
    coord.join(threads)

    new_graph = graph_util.convert_variables_to_constants(
            sess,
            sess.graph_def,
            output_node_names=['accuracy'])
    tf.train.write_graph(new_graph, '', 'graph.pb', as_text=False)


step 0, accuracy: 0, loss : 11.2864
step 1, accuracy: 0.894, loss : 0.771091
step 2, accuracy: 0.896, loss : 1.62691
step 3, accuracy: 0.9, loss : 1.58644
step 4, accuracy: 0.892, loss : 1.41257
step 5, accuracy: 0.898, loss : 0.942975
step 6, accuracy: 0.898, loss : 0.506835
step 7, accuracy: 0.902, loss : 0.338646
step 8, accuracy: 0.904, loss : 0.321884
step 9, accuracy: 0.894, loss : 0.469784
0.898
0.898
0.898
0.898
0.898
0.898
0.898
0.898
0.898
0.898
INFO:tensorflow:Froze 10 variables.
Converted 10 variables to const ops.
